In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-14 16:41:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   337KB/s    in 3.2s    

2023-07-14 16:41:22 (337 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
with open("input.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [7]:
print(len(text))

1115394


In [8]:
chars = sorted(list(set(text)))
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
vocab_size = len(chars)
print(vocab_size)

65


In [10]:
string2int = { ch:i for i,ch in enumerate(chars) }
print(string2int["c"])

41


In [11]:
int2string = { i:ch for i,ch in enumerate(chars) }
print(int2string[41])

c


In [12]:
def encode(str):
    return [string2int[c] for c in str]
print(encode("hi there"))

[46, 47, 1, 58, 46, 43, 56, 43]


In [13]:
def decode(tokenList):
    return "".join([int2string[i] for i in tokenList])
print(decode(encode("hi there")))

hi there


In [14]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
n = int(0.9 * len(data))
train_data = data[n:]
val_data = data[:n]

In [16]:
block_size = 8
print(train_data[:block_size+1])

tensor([12,  0,  0, 19, 30, 17, 25, 21, 27])


In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"context: {context} - target: {target}")

context: tensor([12]) - target: 0
context: tensor([12,  0]) - target: 0
context: tensor([12,  0,  0]) - target: 19
context: tensor([12,  0,  0, 19]) - target: 30
context: tensor([12,  0,  0, 19, 30]) - target: 17
context: tensor([12,  0,  0, 19, 30, 17]) - target: 25
context: tensor([12,  0,  0, 19, 30, 17, 25]) - target: 21
context: tensor([12,  0,  0, 19, 30, 17, 25, 21]) - target: 27


### Generalization of the block_size which includes the batch dimension as well (used to keep the GPUs busy)

In [20]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("Inputs:", xb.shape)
print(xb)
print("Targets:", yb.shape)
print(yb)

print("---")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"{context} -> {target}")

Inputs: torch.Size([4, 8])
tensor([[ 6,  1, 52, 53, 58,  1, 58, 47],
        [ 6,  1, 54, 50, 39, 52, 58, 43],
        [ 1, 58, 46, 47, 57,  1, 50, 47],
        [ 0, 32, 46, 43, 56, 43,  1, 42]])
Targets: torch.Size([4, 8])
tensor([[ 1, 52, 53, 58,  1, 58, 47, 50],
        [ 1, 54, 50, 39, 52, 58, 43, 58],
        [58, 46, 47, 57,  1, 50, 47, 60],
        [32, 46, 43, 56, 43,  1, 42, 53]])
---
tensor([6]) -> 1
tensor([6, 1]) -> 52
tensor([ 6,  1, 52]) -> 53
tensor([ 6,  1, 52, 53]) -> 58
tensor([ 6,  1, 52, 53, 58]) -> 1
tensor([ 6,  1, 52, 53, 58,  1]) -> 58
tensor([ 6,  1, 52, 53, 58,  1, 58]) -> 47
tensor([ 6,  1, 52, 53, 58,  1, 58, 47]) -> 50
tensor([6]) -> 1
tensor([6, 1]) -> 54
tensor([ 6,  1, 54]) -> 50
tensor([ 6,  1, 54, 50]) -> 39
tensor([ 6,  1, 54, 50, 39]) -> 52
tensor([ 6,  1, 54, 50, 39, 52]) -> 58
tensor([ 6,  1, 54, 50, 39, 52, 58]) -> 43
tensor([ 6,  1, 54, 50, 39, 52, 58, 43]) -> 58
tensor([1]) -> 58
tensor([ 1, 58]) -> 46
tensor([ 1, 58, 46]) -> 47
tensor([ 1, 58, 

# BigramLanguageModel

Use pytorch to implement a very simple neural network: the bigram language model.

Make predictions with context 1. "If I am token number 5, what comes next?"

Use negative log likelihood to measure loss.

In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # Create the logits from the embedding table
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            # Reshape to fit the cross-entropy function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # Compute the loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
output = m.generate(idx, max_new_tokens=100)[0].tolist()
print(decode(output))

torch.Size([32, 65])
tensor(4.4913, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [33]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [25]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # Create the logits from the embedding table
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            # Reshape to fit the cross-entropy function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # Compute the loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [29]:
batch_size = 32

losses = []

for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    losses.append(loss)

print(loss.item())

2.3153867721557617


In [31]:
idx = torch.zeros((1,1), dtype=torch.long)
output = m.generate(idx, max_new_tokens=100)[0].tolist()
print(decode(output))




PENDATI and hXsoughicer fare r

CENSPRUMI.
PRUMIO, binouricirre ncido she s wavavere f'd ot ory t.
Bukinco'therelinorin ansous airsthethove inthec melllloine spelllorer, oout hope,
Sit s.
GO: hak! r!
A t amam h t ghe,
Bato ed bdr at, f, thast to Is he hiso se:
Grouled per la memp y me avey p. ze it pped s hy to sikng
en' pthis whaveewit the thithetuturd IOSine ma osanastormik y mupe s th y.
o cest:


Mielatoowis,Sy has; ls swht y and IO:
UCENA bey y youinou wig by, mak'g t lea ere wer pedagon astof inill Pes?
GRAn avener my kend ifrer ait cel nfy nouepem.
beaished adand smangscke c vethes fond.

Kak

APETERONDETI youste n yor hean h ans pofin t our oucene.
GRAn t as y, wif we:
IO:
IEve; beld bee:
Kas mee fayou y ape p muc, beyo, f d atout'd frslefooinde, fe'd nddonole nte aisemeeth s coway tie.

BAd:d.

bl f torraze?
Whe chre rr l'des an giou sstedad aterpe ow te! ak;


TERUCH:



Wierevecheer tr st ainjas t mesisheg bor mear.
BA:
fin ch wo ay manchithyo'tlerionghond me scan'eat: ma

# Mathematical trick in self-attention

In [49]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [55]:
# We want x[b,t] = mean_{i <= t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [56]:
wei = torch.tril(torch.ones(T, T)) # weights
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) BATCH MATRIX MULTIPLY
torch.allclose(xbow, xbow2)

True

In [57]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [58]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [52]:
torch.tril(torch.ones((3,3)))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [53]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


# Self-attention

In [61]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

# out = wei @ x
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [63]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)